In [1]:
import ciropt as co
import cvxpy as cp

import PEPit
import PEPit.functions as pep_func
import ciropt.function as co_func
import sympy as sp

In [2]:
L_smooth = 1.
mu = 0 #0.1
Capacitance = 1.


solver = "ipopt"
# solver = "ipopt_qcqp"
# solver = "ipopt_qcqp_matrix"

# Ciropt problem

In [3]:
def gradient_flow_circuit(mu, L_smooth, params=None): 
    if params is not None:
        # verification mode: PEP
        problem = PEPit.PEP()
        package = pep_func
        h, alpha, beta, eta, C, invC = params["h"], params["alpha"], params["beta"], params["eta"], params["C"], params["invC"]
    else:
        # Ciropt mode
        problem = co.CircuitOpt()
        package = co_func
        C = sp.symbols('C')
        invC = sp.symbols("invC")
        problem.discretization_params = sorted(problem.discretization_params + ["C", "invC"])
        h, alpha, beta, eta = problem.h, problem.alpha, problem.beta, problem.eta
    func = co.define_function(problem, mu, L_smooth, package )
    x_star, y_star, f_star = func.stationary_point(return_gradient_and_function_value=True)

    x_1 = problem.set_initial_point()
    y_1, f_1 = func.oracle(x_1)

    x_1p5 = x_1 - (alpha * h * invC) * y_1 
    y_1p5, f_1p5 = func.oracle(x_1p5)

    x_2 = x_1  - (beta * h * invC) * y_1 - ((1 - beta) * h * invC) * y_1p5
    y_2, f_2 = func.oracle(x_2)

    E_1 = (C/2) * (x_1 - x_star)**2
    E_2 = (C/2) * (x_2 - x_star)**2
    Delta_1 = eta * (f_1 - f_star)
    problem.set_performance_metric(E_2 - (E_1 - Delta_1))
    return problem

In [4]:
problem = gradient_flow_circuit(mu, L_smooth)
problem.obj = problem.eta

bounds = {"C":{"ub": 10, "lb":1e-3}}

res, model = problem.solve(solver=solver, bounds=bounds, verbose=False)[:2]

dim_G=5, dim_F=4
Ipopt total # of variables = 71
Actual # of variables = 40


In [5]:
res

{'C': 10.00000009909091,
 'alpha': 1.1399172342650544e-05,
 'beta': 0.02629254510698183,
 'eta': 10.000000094750202,
 'h': 10.000226267838396,
 'invC': 0.09999999900909094,
 'rho': 20.284120306168266}

# PEP verification

In [6]:
problem = gradient_flow_circuit(mu, L_smooth, params=res)
diff = problem.solve(verbose=0, solver=cp.MOSEK)
print(f"{diff=}")

diff=2.7442957971546286e-09
